In [2]:
import pandas as pd
import numpy as np
import sklearn

In [3]:
print(pd.__version__)
print(np.__version__)
print(sklearn.__version__)

2.2.0
1.26.4
1.3.0


In [7]:
pip install numpy==1.26.4

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for numpy==1.26.4 from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.0 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB 1.4 MB/s eta 0:00:12
   ---------------------------------------- 0.1/15.8 MB 1.1 MB/s eta 0:00:15
   ---------------------------------------- 0.1/15.8 MB 1.1 MB/s eta 0:00:15
   ---------------------------------------- 0.2/15.8 MB 984.6 kB/s eta 0:00:16
   ---------------------------------------- 0.2/15.8 MB 984.6 kB/s eta 0:00:16
    --------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires grpcio>=1.48.2, which is not installed.
tensorflow-intel 2.15.0 requires grpcio<2.0,>=1.24.3, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.


In [9]:
import numpy as np

In [10]:
print(np.__version__)

1.24.3


In [12]:
pip install scikit-learn==1.3.0

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [4]:
print(sklearn.__version__)

1.3.0


In [5]:
df = pd.read_csv("college_data.csv")

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

In [7]:
encoded_features = {}

for feature in ['branch', 'gender', 'category', 'secondary_seat_type', 'score_type', 'city'] :
    df[feature] = label_encoder.fit_transform(df[feature])
    #storing the mappings in a dictionary
    encoded_features[feature] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [8]:
#encode target variable
df['college_name'] = label_encoder.fit_transform(df['college_name'])

#storing the mappings in a dictionary
encoded_features['college_name'] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [9]:
#Define feature and target variable

X = df.drop('college_name', axis=1)
y = df['college_name']

In [10]:
from sklearn.model_selection import train_test_split

#Split data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
def encode_input_data(input_data, mappings):
    encoded_data = input_data.copy()
    for feature in mappings:
        if feature in encoded_data:
            encoded_data[feature] = encoded_data[feature].map(mappings[feature])
    return encoded_data

In [12]:
input_data = pd.DataFrame({
    'percentile': [100],
    'branch': ['computer science and engineering'],
    'gender': ['m'],
    'category': ['open'],
    'secondary_seat_type': ['all india seats allotted to all india candidature candidates with jee(main) score'],
    'score_type': ['mht-cet'],
    'city': ['pune']
})

In [13]:
from sklearn.ensemble import RandomForestClassifier
rfmodel = RandomForestClassifier(random_state=42)
rfmodel.fit(X_train, y_train)
accuracy = rfmodel.score(X_test, y_test)
print(accuracy)

0.5603605331287755


In [ ]:
# from sklearn.linear_model import LogisticRegression

# # Initialize the multinomial logistic regression model with different hyperparameters
# logreg_model = LogisticRegression(multi_class='multinomial', solver='saga', C=10, max_iter=1000, random_state=42)

# # Train the model
# logreg_model.fit(X_train, y_train)

# # Test the model
# accuracy = logreg_model.score(X_test, y_test)
# print(accuracy)

In [ ]:
# #gb
# from sklearn.ensemble import GradientBoostingClassifier

# # Initialize the GradientBoostingClassifier with default parameters and random_state
# gb_model = GradientBoostingClassifier(random_state=42)

# # Train the model
# gb_model.fit(X_train, y_train)

# # Evaluate the model
# accuracy = gb_model.score(X_test, y_test)
# print("Accuracy:", accuracy)

In [14]:
encoded_input = encode_input_data(input_data, encoded_features)
encoded_input

,percentile,branch,gender,category,secondary_seat_type,score_type,city
0,100,21,1,47,0,2,65


In [15]:
#Make predictions for the encoded_input_data using trained Random Forest Model
encoded_predictions_rf = rfmodel.predict(encoded_input)
encoded_predictions_rf

array([203])

In [16]:
#Make predictions for the encoded_input_data using trained LogisticRegression Model
encoded_predictions_lrm = logreg_model.predict(encoded_input)
encoded_predictions_lrm

NameError: name 'logreg_model' is not defined

In [17]:
encoded_predictions_proba_rf = rfmodel.predict_proba(encoded_input)

top_5_indices_rf = encoded_predictions_proba_rf.argsort()[:, -5:][0]

top_5_indices_rf

array([ 27, 272,  38,  56, 203], dtype=int64)

In [ ]:
#lrm
encoded_predictions_proba_lrm = logreg_model.predict_proba(encoded_input)

top_5_indices_lrm = encoded_predictions_proba_lrm.argsort()[:, -5:][0]

top_5_indices_lrm

In [18]:
top_5_college_labels_rf = [list(encoded_features['college_name'].keys())[idx] for idx in top_5_indices_rf]

top_5_probabilities_rf = encoded_predictions_proba_rf[0][top_5_indices_rf]

top_5_college_labels_rf

["bansilal ramnath agarawal charitable trust's vishwakarma institute of technology, bibwewadi, pune",
 "sinhgad technical education society's smt. kashibai navale college of engineering,vadgaon,pune",
 'coep technological university',
 "dr. d. y. patil pratishthan's d.y.patil college of engineering akurdi, pune",
 'pune institute of computer technology, dhankavdi, pune']

In [ ]:
#lrm
top_5_college_labels_lrm = [list(encoded_features['college_name'].keys())[idx] for idx in top_5_indices_lrm]

top_5_probabilities_lrm = encoded_predictions_proba_lrm[0][top_5_indices_lrm]

top_5_college_labels_lrm

In [19]:
top_5_probabilities_perc_rf = top_5_probabilities_rf*100
top_5_probabilities_perc_rf

array([ 2., 12., 13., 26., 40.])

In [22]:
import joblib

In [23]:
joblib.dump(rfmodel, 'rfmodel.pkl')

['rfmodel.pkl']